### **PROJECT-4. Решение комплексной бизнес-задачи**

*Мельничихина Евгения Андреевна, DAPR_171*



Перед нами поставлена задача: повысить средний чек продажи обучающих курсов. Для этого необходимо подготовить основу рекомендательной системы, а именно таблицу с рекомендованными наиболее популярными курсами.




In [2]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
from collections import Counter
from itertools import combinations 
from pprint import pprint

Пишем SQL запрос к базе данных, выгружаем id пользователей, купивших более одного курса, а также id купленных ими курсов.

In [3]:
def get_data(): 
    query='''with filtered_data as
    (select
    user_id,
    count(distinct resource_id) as resource_count
    from final.carts
    join final.cart_items on carts.id=cart_items.cart_id
    where state='successful' and resource_type='Course'
    group by 1
    having count(distinct resource_id) > 1
    order by 1)
    select
    distinct filtered_data.user_id,
    cart_items.resource_id
    from filtered_data
    join final.carts on filtered_data.user_id=carts.user_id
    join final.cart_items on cart_items.cart_id = carts.id
    where state='successful' and resource_type='Course'
    order by 1, 2''' 
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor) 
    dict_cur.execute(query) 
    rows = dict_cur.fetchall() 
    data = [] 
    for row in rows: 
        data.append(dict(row)) 
    return data

data = get_data()
df = pd.DataFrame(data)
df.head()


,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [4]:
# Изучаем получившийся датафрейм
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


In [5]:
# Выводим количество уникальных пользователей, купивших более одного курса
print(df['user_id'].nunique())

# Выводим количество уникальных курсов
print(df['resource_id'].nunique())

12656
126


Как мы помним, изначально было 127 уникальных курсов, но один из них был в итоге отфильтрован, так как никто из наших пользователей его не приобрел.

In [6]:
# Группируем данные в разрезе пользователей, считаем количество уникальных купленных курсов на пользователя, выводим id этих курсов. 
df_grouped = df.groupby('user_id')['resource_id'].agg(['nunique', set]).sort_values(by='nunique', ascending=False).reset_index()
df_grouped

,user_id,nunique,set
0,694146,19,"{515, 1156, 776, 523, 907, 908, 909, 670, 552,..."
1,953401,18,"{514, 515, 741, 518, 519, 551, 552, 490, 523, ..."
2,1050532,18,"{864, 551, 552, 489, 679, 363, 523, 776, 366, ..."
3,992470,17,"{514, 515, 517, 518, 519, 551, 489, 552, 523, ..."
4,722252,17,"{514, 515, 357, 551, 489, 490, 523, 829, 908, ..."
...,...,...,...
12651,989104,2,"{368, 514}"
12652,989634,2,"{571, 357}"
12653,989824,2,"{504, 551}"
12654,989950,2,"{757, 367}"


In [8]:
# Делим курсы по парам 
course_pairs = df_grouped['set'].apply(lambda x:list(combinations(sorted(x), 2)))

# Создаем список из пар курсов
courses_pairs_list = []
for pair_course in course_pairs.values:
    for course in pair_course:
        courses_pairs_list.append(course)
   
pprint(courses_pairs_list[:10])

[(489, 490),
 (489, 515),
 (489, 523),
 (489, 552),
 (489, 564),
 (489, 566),
 (489, 670),
 (489, 764),
 (489, 776),
 (489, 809)]


In [11]:
# Считаем количество повторений одинаковых пар
pairs_count = Counter(courses_pairs_list)
pairs_count_dict = dict(pairs_count)
pairs_count

Counter({(551, 566): 797,
         (515, 551): 417,
         (489, 551): 311,
         (523, 551): 304,
         (566, 794): 290,
         (489, 515): 286,
         (490, 566): 253,
         (490, 551): 247,
         (570, 752): 247,
         (569, 572): 216,
         (515, 523): 213,
         (553, 745): 212,
         (489, 523): 206,
         (569, 840): 204,
         (514, 551): 200,
         (516, 745): 199,
         (515, 566): 195,
         (489, 566): 188,
         (504, 572): 184,
         (572, 840): 178,
         (551, 552): 177,
         (507, 570): 172,
         (490, 809): 163,
         (489, 490): 152,
         (507, 752): 150,
         (523, 552): 144,
         (490, 515): 143,
         (551, 570): 142,
         (514, 515): 139,
         (504, 569): 139,
         (514, 566): 138,
         (551, 745): 138,
         (502, 551): 135,
         (504, 840): 135,
         (571, 1125): 122,
         (523, 566): 120,
         (502, 566): 120,
         (570, 809): 119,
         (7

Наиболее часто встречаемая пара: (551, 566)

In [ ]:
# Выводим количество уникальных пар курсов
print(len(pairs_count))

3989


In [ ]:
# Выводим единичные курсы по их популярности
most_popular_courses  = Counter(df['resource_id'])
most_popular_courses

Counter({551: 2935,
         566: 2342,
         515: 1311,
         489: 1125,
         490: 1084,
         523: 1053,
         514: 983,
         794: 875,
         745: 829,
         570: 720,
         502: 686,
         809: 639,
         504: 616,
         840: 614,
         507: 603,
         552: 590,
         572: 589,
         571: 585,
         752: 548,
         569: 545,
         564: 504,
         764: 485,
         516: 425,
         519: 419,
         1103: 416,
         553: 379,
         679: 372,
         1100: 362,
         357: 360,
         750: 356,
         1125: 353,
         356: 351,
         777: 347,
         749: 344,
         503: 308,
         366: 294,
         562: 282,
         363: 263,
         361: 258,
         757: 237,
         829: 229,
         513: 228,
         908: 221,
         776: 220,
         568: 214,
         664: 212,
         741: 210,
         517: 208,
         1115: 204,
         765: 196,
         511: 192,
         672: 185,
  

In [ ]:
# Делаем преобразования над файлом, чтобы дальше с ним работать
most_popular_courses_dict = dict(most_popular_courses)
most_popular_courses_list = sorted(most_popular_courses_dict.items(), key=lambda x: -x[1])

In [ ]:
# Пишем функцию для создания списка уникальных курсов от самого популярного к наименее популярному
def all_courses_sorted(data):
    all_courses_list = list()
    for i in data:
        if i[0] not in all_courses_list:
            all_courses_list.append(i[0])
    return all_courses_list

all_courses = all_courses_sorted(most_popular_courses_list)
print(all_courses)
print(len(all_courses))

[551, 566, 515, 489, 490, 523, 514, 794, 745, 570, 502, 809, 504, 840, 507, 552, 572, 571, 752, 569, 564, 764, 516, 519, 1103, 553, 679, 1100, 357, 750, 1125, 356, 777, 749, 503, 366, 562, 363, 361, 757, 829, 513, 908, 776, 568, 664, 741, 517, 1115, 765, 511, 672, 367, 659, 791, 508, 509, 756, 912, 368, 1138, 1102, 1161, 743, 518, 753, 1099, 814, 1140, 909, 563, 742, 1141, 1144, 1152, 1116, 907, 1139, 1101, 1128, 1184, 358, 670, 1187, 1104, 1185, 1156, 1186, 744, 863, 755, 1145, 1124, 1129, 864, 359, 1147, 813, 1181, 810, 1188, 671, 360, 1146, 362, 803, 862, 800, 364, 365, 865, 835, 837, 1160, 866, 834, 1198, 836, 830, 911, 1182, 902, 833, 1199, 1201, 1200]
126


In [ ]:
# Пишем функцию для рекомендаций двух наиболее часто встречающихся курсов (для всех курсов без ограничения по частотности) 
def recommendation(pairs_count_dict, all_courses):
    rec_1 = []
    rec_2 = []
    for i in all_courses:
        temp_dict = dict()
        for j in pairs_count_dict:
            if i in j:
                temp_dict[j] = pairs_count_dict[j]
        sorting_courses = sorted(temp_dict.items(), key=lambda x:-x[1])
        sorting_courses = sorting_courses[:2]
        if sorting_courses[0][0][0] != i:
            rec_1.append(sorting_courses[0][0][0])
        else:
            rec_1.append(sorting_courses[0][0][1])
        if sorting_courses[1][0][0] != i:
            rec_2.append(sorting_courses[1][0][0])
        else:
            rec_2.append(sorting_courses[1][0][1])
    return rec_1, rec_2

# Создаем два списка с рекомендованными курсами
list_of_1_recommendation, list_of_2_recommendation = recommendation(pairs_count_dict, all_courses)

# Проверяем длину списков
print(len(list_of_1_recommendation))
print(len(list_of_2_recommendation))   
               
    
                

126
126


Создаем таблицу рекомендаций, в которой в первом столбце перечислены все уникальные курсы по убыванию их популярности, во втором столбце указан самый популярный, наиболее часто встречаемый в парах рекомендованный курс, в третьем столбце указан второй по популярности рекомендованный курс. Данная таблица составлена БЕЗ учета ограничения частотности пар

In [ ]:
recommendation_table1 = pd.DataFrame(data={'all_courses': all_courses, 'first_course_recommendation': list_of_1_recommendation, 'second_course_recommendation': list_of_2_recommendation})
recommendation_table1.head(15)

,all_courses,first_course_recommendation,second_course_recommendation
0,551,566,515
1,566,551,794
2,515,551,489
3,489,551,515
4,490,566,551
5,523,551,515
6,514,551,515
7,794,566,551
8,745,553,516
9,570,752,507


Но, поскольку некоторые пары встречаются довольно редко, мы можем определить минимальный порог частотности и отфильтровать из датафрейма значения ниже этого порога. 

In [ ]:
# Создаем датафрейм с уникальными парами курсов и их частотностью
pairs_count_df = pd.DataFrame(data={'pair_of_courses': pairs_count_dict.keys(), 'freq': pairs_count_dict.values()})
pairs_count_df = pairs_count_df.sort_values(by='freq', ascending=False).reset_index(drop=True)
pairs_count_df['percent'] = round(pairs_count_df['freq'] / 3989 * 100, 1)
pairs_count_df.head()

,pair_of_courses,freq,percent
0,"(551, 566)",797,20.0
1,"(515, 551)",417,10.5
2,"(489, 551)",311,7.8
3,"(523, 551)",304,7.6
4,"(566, 794)",290,7.3


In [ ]:
# Находим минимальный порог частотности
min_freq = np.percentile(pairs_count_df['freq'],85)
min_freq

15.0

In [ ]:
# Создаем отфильтрованный датафрейм без пар, не проходящих минимальный порог частотности
min_freq_df = pairs_count_df[pairs_count_df['freq'] >= min_freq]
min_freq_df.head()

,pair_of_courses,freq,percent
0,"(551, 566)",797,20.0
1,"(515, 551)",417,10.5
2,"(489, 551)",311,7.8
3,"(523, 551)",304,7.6
4,"(566, 794)",290,7.3


In [ ]:
# На основе отфильтрованного выше датафрейма создаем словарь
min_freq_dict = dict()
for i, num in enumerate(min_freq_df['pair_of_courses'].values):
    min_freq_dict[num] = min_freq_df['freq'][i]

min_freq_dict

{(551, 566): 797,
 (515, 551): 417,
 (489, 551): 311,
 (523, 551): 304,
 (566, 794): 290,
 (489, 515): 286,
 (490, 566): 253,
 (490, 551): 247,
 (570, 752): 247,
 (569, 572): 216,
 (515, 523): 213,
 (553, 745): 212,
 (489, 523): 206,
 (569, 840): 204,
 (514, 551): 200,
 (516, 745): 199,
 (515, 566): 195,
 (489, 566): 188,
 (504, 572): 184,
 (572, 840): 178,
 (551, 552): 177,
 (507, 570): 172,
 (490, 809): 163,
 (489, 490): 152,
 (507, 752): 150,
 (523, 552): 144,
 (490, 515): 143,
 (551, 570): 142,
 (514, 515): 139,
 (504, 569): 139,
 (514, 566): 138,
 (551, 745): 138,
 (504, 840): 135,
 (502, 551): 135,
 (571, 1125): 122,
 (523, 566): 120,
 (502, 566): 120,
 (570, 809): 119,
 (752, 809): 115,
 (490, 523): 114,
 (357, 571): 112,
 (523, 564): 110,
 (551, 749): 109,
 (551, 777): 107,
 (516, 553): 107,
 (551, 679): 104,
 (551, 564): 103,
 (515, 749): 103,
 (356, 571): 103,
 (568, 745): 102,
 (356, 357): 100,
 (363, 511): 99,
 (551, 571): 98,
 (551, 809): 96,
 (502, 514): 95,
 (551, 794): 

In [ ]:
# Вносим некоторые изменения в функцию для создания списков рекомендаций курсов
def recommendation2(min_freq_dict, all_courses):
    rec_1 = []
    rec_2 = []
    for i in all_courses:
        temp_dict = dict()
        for j in min_freq_dict:
            if i in j:
                temp_dict[j] = min_freq_dict[j]
        sorting_courses = sorted(temp_dict.items(), key=lambda x:-x[1])
        sorting_courses = sorting_courses[:2]
        try:
            if sorting_courses[0][0][0] != i:
                rec_1.append(sorting_courses[0][0][0])
            if sorting_courses[0][0][1] != i:
                rec_1.append(sorting_courses[0][0][1])
        except IndexError:
            rec_1.append(551)
        try:
            if sorting_courses[1][0][0] != i:
                rec_2.append(sorting_courses[1][0][0])
            if sorting_courses[1][0][1] != i:
                rec_2.append(sorting_courses[1][0][1])
        except IndexError:
            rec_2.append(566)
    return rec_1, rec_2


list1, list2 = recommendation2(min_freq_dict, all_courses)

# Проверяем длину списков
print(len(list1))
print(len(list2))


126
126


In [ ]:
# Создаем финальную таблицу с рекомендациями
recommendation_table2 = pd.DataFrame(data={'all_courses': all_courses, 'recommendation_1': list1, 'recommendation_2': list2})
recommendation_table2.head(15)

,all_courses,recommendation_1,recommendation_2
0,551,566,515
1,566,551,794
2,515,551,489
3,489,551,515
4,490,566,551
5,523,551,515
6,514,551,515
7,794,566,551
8,745,553,516
9,570,752,507


Я также провела промежуточное исследование: если при выполнении функции recommendation2 не хватало пар курсов из-за проведенной ранее фильтрации, то в списки вместо них добавлялись нули. В строках с нулями отсутствовали два самых популярных курса - 551 и 566. Поэтому я посчитала, что пропуски можно заполнить этими курсами напрямую в самой функции. 
На мой взгляд, для курсов с низкой частотностью правильнее было бы давать рекомендации курсов, смежных по специализации или каким-либо образом взаимосвязанных. Но, поскольку у нас нет наименований курсов, а только их id, я считаю, что, в таком случае будет неплохим вариантом советовать пару курсов бестселлеров.